In [1]:
import numpy as np
import numba as nb

import multiprocessing as mp

import matplotlib.pyplot as plt
import matplotlib
from time import clock
from scipy.optimize import curve_fit
from scipy.optimize import minimize
import os

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

from pre_fun import *

import matplotlib.ticker as ticker
matplotlib.rcParams['text.usetex'] = False
# matplotlib.rcParams['text.latex.unicode'] = True

In [2]:
# Impedance Simulation

stp = Setup(  I = 500,
                    N = 2 ** 18,
                    Dt = 1e-4,
                    T = 300,
                    L = 10,
                    lengthscale = 1e9,
                    c0_in = 1.0,
                    DA = 1.0,
                    DC = 1.0,
                    D0_in = 1e-10,
                    epsilon = 1.0,
                    epsilon_m = 80.0,
                    testname = '20180603_model_1_impedance_1',
                    model = 1,
                   sim_method = "c_only_imp",
                   kA = 1e-3,
                   kC = 1e-3,
                   foxA = 0.5e-3,
                       foxC = 0.5e-3, 
                    steady_state_tol = 5e-7,)

solver(stp)

Start Impedance - current only - Simulation
Min Freq, Frequency Resolution Df [Hz]:  38146.97265625
Min Freq, Frequency Resolution Df [None]:  0.03814697265625
Maximal Frequency, Nyquist [Hz]:  4545454545.454545
Maximal Frequency, Nyquist [None]:  4545.454545454545
Number of Points:  262144
Total Amplitude [mV]:  18.804376084000893
Start 'c_only' Simulation
Time Step:  1
Time Step:  2
Time Step:  3
Time Step:  4
Time Step:  5
Time Step:  6
Time Step:  7
Time Step:  8
Time Step:  9
Time Step:  10
Time Step:  11
Time Step:  12
Time Step:  13
Time Step:  14
Time Step:  15
Time Step:  16
Time Step:  17
Time Step:  18
Time Step:  19
Time Step:  20
Time Step:  30
Time Step:  40
Time Step:  50
Time Step:  60
Time Step:  70
Time Step:  80
Time Step:  90
Time Step:  5000
Time Step:  10000
Time Step:  15000
Time Step:  20000
Time Step:  25000
Time Step:  30000
Time Step:  35000
Time Step:  40000
Time Step:  45000
Time Step:  50000
Time Step:  55000
Time Step:  60000
Time Step:  65000
Time Step: 

In [ ]:
# Sweep Simulation
# 
sol_list = []
voltage_list = [-30e-3, -20e-3, -15e-3, -10e-3 ,-7.5e-3 -5e-3, -2.5e-3, -1e-3, -0.5e-3,
                0.5e-3, 1e-3, 2.5e-3, 5e-3, 10e-3, 15e-3, 20e-3, 30e-3 ]


for i in range( 0,voltage_list.__len__() ):
    print(i)
    stp_name = '20180602_' + 'test_' + str(i) + '_model_1'
    stp = Setup(  I = 500,
                    N = 2 ** 16,
                    Dt = 1e-4,
                    T = 300,
                    L = 10,
                    lengthscale = 1e9,
                    c0_in = 1.0,
                    DA = 1.0,
                    DC = 1.0,
                    D0_in = 1e-10,
                    epsilon = 1.0,
                    epsilon_m = 80.0,
                    testname = stp_name,
                    model = 1,
                   sim_method = "single_const",
                   kA = 1e-3,
                   kC = 1e-3,
                   foxA = 0.5e-3,
                       foxC = 0.5e-3, 
                    steady_state_tol = 5e-7,)

    # set pertubation voltage
    stp.set_input_voltage_data( voltage_list[i] )

    # call solver
    solver( stp )
    
    
del stp

In [ ]:
# sol_list_high.append(sol_list[-2])

In [ ]:
class singleSineSim():
    
    # fit function
    def fitCurrent( self, independent, y0, k0, k1 ):
    """
    Independent input parameters, list:
        
        independent[0] = time axis
        independent[1] = frequency
        independent[2] = reference time T0
        
    """
    
    time = independent[0]
    freq = independent[1]
    T0 = independent[2]
    
    return y0 + k0 * np.sin( 2 * np.pi * freq * time *T0 ) + k1 * np.cos( 2 * np.pi * freq * time *T0 )
    
    # fitting
    def fitting( self, ):
        popt, pcov = curve_fit( fitCurrent, [t[1:],freq,T0], current[0,1:],  )
   
    
        # calculate parameters for current
        A = np.sqrt( popt[1] ** 2 + popt[2] ** 2 ) * f0 * _ELE_CHARGE * 1e17
        phase = np.arctan( popt[2] / popt[1])
        
        # calculate impedance
        Z = np.abs( U0 * phi0 ) * np.exp(1j * phase) / np.abs(A)
        
        return
        
    
    
    
    # saving
    if os.path.isfile(testname):
        with open(testname, mode = "a") as file_id:

            np.savetxt(file_id, savearray, delimiter = ",", newline = "\n")

    else:
        with open(testname, mode = "w") as file_id:

                    np.savetxt(file_id, savearray, delimiter = ",", newline = "\n")
    
    # plotting
    fig = plt.figure( dpi = 120 )

    ax = fig.add_subplot(1,1,1)

    # input voltage
    ax.plot(t[1:], phiC[1:], color = "blue")

    # anodic current
    ax.plot(t[1:], current[0,1:], color = "red")

    # plot fitted current
    ax.plot( t[1:], fitCurrent( [t[1:], freq, T0], popt[0], popt[1], popt[2] ) , ls = "--", color = "black")


    plt.show()
    

In [ ]:
# fit function
def fitCurrent( independent, y0, k0, k1 ):
    """
    Independent input parameters, list:

        independent[0] = time axis
        independent[1] = frequency
        independent[2] = reference time T0

    """

    time = independent[0]
    freq = independent[1]
    T0 = independent[2]

    return y0 + k0 * np.sin( 2 * np.pi * freq * time *T0 ) + k1 * np.cos( 2 * np.pi * freq * time *T0 )


# # fitting
#popt, pcov = curve_fit( fitCurrent, [t[1:],freq,T0], current[0,1:]  )
   
    
# calculate parameters for current
#A = np.sqrt( popt[1] ** 2 + popt[2] ** 2 ) * f0 * _ELE_CHARGE * 1e17
#phase = np.arctan( popt[2] / popt[1])

# calculate impedance
#Z = np.abs( U0 * phi0 ) * np.exp(1j * phase) / np.abs(A)


 

#A = popt[1]
#phase = popt[2]



# print("A", A)
# print("phase", np.rad2deg(phase))
# print("Impedance", Z)

# saving impedance results
#savearray = np.array([[ freq, Z, Dt, N, I, simtime  ]])

# plotting
fig = plt.figure( dpi = 120 )

ax = fig.add_subplot(1,1,1)

# input voltage
ax.plot(t[1:], phiC[1:], color = "blue")

# current on different axis
ax1 = ax.twinx()

# anodic current

ax1.plot(t[1:], current2[0,1:], color = "green")
ax1.plot(t[1:], current3[0,1:], color = "green")
ax1.plot(t[1:], current4[0,1:], color = "green")
ax1.plot(t[1:], current1[0,1:], color = "red")
# plot fitted current
#ax1.plot( t[1:], fitCurrent( [t[1:], freq, T0], popt[0], popt[1], popt[2] ) , ls = "--", color = "black")


plt.show()

In [ ]:
Fcurrent1 = np.fft.fft( current1[0,:] )[0:int(stp2.N/2)]
Fcurrent2 = np.fft.fft( current2[0,:] )[0:int(stp2.N/2)]
Fcurrent3 = np.fft.fft( current3[0,:] )[0:int(stp2.N/2)]
Fcurrent4 = np.fft.fft( current4[0,:] )[0:int(stp2.N/2)]

# plotting
fig = plt.figure( dpi = 120 )

ax = fig.add_subplot(1,1,1)

#ax.plot( freq_ax, Fcurrent1.real, color = "blue" )
ax.plot( freq_ax, Fcurrent1.real, color = "red" )
#ax.plot( freq_ax, Fcurrent2.real, color = "black", ls = "--" )
ax.plot( freq_ax, Fcurrent2.real, color = "blue", ls = "-" )
ax.plot( freq_ax, Fcurrent3.real, color = "black", ls = "-" )
#ax.plot( freq_ax, Fcurrent4.real, color = "green", ls = "-" )

In [ ]:
# saving
if os.path.isfile(testname):
    with open(testname, mode = "a") as file_id:
        
        np.savetxt(file_id, savearray, delimiter = ",", newline = "\n")
        
else:
    with open(testname, mode = "w") as file_id:
        
                np.savetxt(file_id, savearray, delimiter = ",", newline = "\n")